<a href="https://colab.research.google.com/github/mtm-007/Recommeder-Systems/blob/main/colabrative_filterig_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.collab import *
from fastai.tabular.all import *
set_seed(42)

In [2]:
path = untar_data(URLs.ML_100k)

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [15]:
last_skywalker = np.array([0.98,0.9,-0.9])
user1 = np.array([0.9,0.8,-0.6])
casablanca = np.array([-0.99,-0.3,0.8])

In [11]:
(last_skywalker*user1).sum(axis=0)

2.1420000000000003

In [17]:
(user1*casablanca).sum()

-1.611

In [18]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [19]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


### DataLoaders

In [24]:
CollabDataLoaders??

In [23]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

/usr/local/lib/python3.10/dist-packages/fastai/torch_core.py:263: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  return getattr(torch, 'has_mps', False)


,user,title,rating
0,271,Michael (1996),4
1,868,Threesome (1994),2
2,264,Rocket Man (1997),2
3,606,Phenomenon (1996),4
4,417,Addams Family Values (1993),3
5,781,Young Guns (1988),3
6,409,Bonnie and Clyde (1967),2
7,223,Apollo 13 (1995),4
8,189,"Blues Brothers, The (1980)",2
9,764,Willy Wonka and the Chocolate Factory (1971),4


In [25]:
n_user = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_user,n_factors)
movie_factors = torch.randn(n_movies,n_factors)
